In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsa
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from transformers import TrainingArguments, Trainer
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sentence_transformers import SentenceTransformer

import pickle

## Модель классификации

In [9]:
with open('../data/data_proc.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
data.describe()

,title,description,tags,tags_list,first_level_tags,second_level_tags,combined_text,lemmatized_text,combined_vector
count,1048,1048,1048,1048,1048,1048,1048,1048,1048
unique,1043,1007,234,234,101,110,1044,1043,1048
top,Смешная история | Выпуск 2,"Подписывайся, чтобы не пропустить!",Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура],[Юмор и сатира],Смешная история | Выпуск 2 Лжедмитрий устраива...,роман юнус блогер кирилл нечаев метать копьё п...,"[-0.6948106, -1.2746366, 0.22210847, -0.715470..."
freq,2,21,224,224,278,260,2,2,1


In [11]:
taxonomy = pd.read_csv("../data/IAB_tags.csv")

# Создаем словарь для соответствия тегов первого уровня тегам 2 уровня
tag_mapping = {}

for _, row in taxonomy.iterrows():
    first_level = row['Уровень 1 (iab)']
    second_level = row['Уровень 2 (iab)']
    if first_level not in tag_mapping:
        tag_mapping[first_level] = set()
    if pd.notna(second_level):
        tag_mapping[first_level].add(second_level)

In [12]:
# Извлечение всех уникальных тегов первого уровня
all_first_level_tags = set(tag_mapping.keys())

# Извлечение всех уникальных тегов второго уровня
all_second_level_tags = set([tag for tags in tag_mapping.values() for tag in tags])

In [13]:
# Бинаризация меток для первого уровня
mlb_first_level = MultiLabelBinarizer(classes=list(all_first_level_tags))
y_first_level = mlb_first_level.fit_transform(data['first_level_tags'])  # Размер: (n_samples, n_first_level_classes)

# Бинаризация меток для второго уровня
mlb_second_level = MultiLabelBinarizer(classes=list(all_second_level_tags))
y_second_level = mlb_second_level.fit_transform(data['second_level_tags'])  # Размер: (n_samples, n_second_level_classes)

# Создание массива индексов видео
video_ids = data.index.values  # Получаем индексы видео

c:\Users\avo_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['', 'Игры и головоломки', 'Компьютеры и цифровые технологии', 'Красота', 'Медицинские направления', 'Наука\tКосмос и астрономия', 'Образование\tОнлайн-образование', 'Отношения знаменитостей', 'События и достопримечательности\tКонцерты и музыкальные мероприятия', 'Создание контента', 'Хобби и стиль', 'Экономика'] will be ignored
  warnings.warn(
c:\Users\avo_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['Валюты', 'Воспитание детей% Воспитание малышей', 'Карточные игры', 'Макияж', 'Психиатрия', 'Путешествия на машине', 'Фильмы и анимация', 'астрология'] will be ignored
  warnings.warn(


In [14]:
# Определение Dataset
class TagDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = torch.tensor(self.encodings[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return item, label

    def __len__(self):
        return len(self.labels)

# Разделение данных на обучающую и валидационную выборки для первого уровня
X_train, X_val, y_train_first, y_val_first, train_video_ids, val_video_ids = train_test_split(
    np.vstack(data['combined_vector'].values),
    y_first_level,
    video_ids,
    test_size=0.2,
    random_state=42
)

# Создание Dataset и DataLoader для первого уровня
train_dataset_first = TagDataset(X_train, y_train_first)
val_dataset_first = TagDataset(X_val, y_val_first)
train_loader_first = DataLoader(train_dataset_first, batch_size=16, shuffle=True)
val_loader_first = DataLoader(val_dataset_first, batch_size=16)

In [15]:
# Определение модели CNN
class TextCNN(nn.Module):
    def __init__(self, num_classes):
        super(TextCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=5, padding=2)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(300, num_classes)  # Финальный слой для классификации

    def forward(self, x):
        x = x.unsqueeze(1)  # Добавляем размер для канала
        x = x.permute(0, 2, 1)  # Меняем размерность для свертки

        # Применение свертки и пулинга
        x1 = torch.relu(self.conv1(x))
        x1 = torch.max_pool1d(x1, x1.size(2)).squeeze(2)

        x2 = torch.relu(self.conv2(x))
        x2 = torch.max_pool1d(x2, x2.size(2)).squeeze(2)

        x3 = torch.relu(self.conv3(x))
        x3 = torch.max_pool1d(x3, x3.size(2)).squeeze(2)

        # Конкатенация и дропаут
        x = torch.cat((x1, x2, x3), 1)
        x = self.dropout(x)
        return self.fc(x)

In [16]:
# Функция для обучения модели
def train_model(model, train_loader, val_loader=None, num_epochs=10):
    criterion = nn.BCEWithLogitsLoss()  # Функция потерь
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Валидация
        if val_loader:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()
            print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}")

# Обучение модели для первого уровня
model_first_level = TextCNN(num_classes=y_first_level.shape[1])
train_model(model_first_level, train_loader_first, val_loader_first, num_epochs=5)

# Предсказания для первого уровня
model_first_level.eval()
predictions_first_level = []
with torch.no_grad():
    for inputs, _ in val_loader_first:
        outputs = model_first_level(inputs)
        predictions_first_level.append((torch.sigmoid(outputs) > 0.2).cpu().numpy())

predictions_first_level = np.vstack(predictions_first_level)


Epoch 1, Validation Loss: 0.12025466242006846
Epoch 2, Validation Loss: 0.10058552718588284
Epoch 3, Validation Loss: 0.0937372472669397
Epoch 4, Validation Loss: 0.09103329266820635
Epoch 5, Validation Loss: 0.08952406101993152


In [17]:
# Разделение данных на обучающую и валидационную выборки для второго уровня
X_train_second, X_val_second, y_train_second, y_val_second, train_video_ids_second, val_video_ids_second = train_test_split(
    np.vstack(data['combined_vector'].values),
    y_second_level,
    video_ids,
    test_size=0.2,
    random_state=42
)

# Создание Dataset и DataLoader для второго уровня
train_dataset_second = TagDataset(X_train_second, y_train_second)
val_dataset_second = TagDataset(X_val_second, y_val_second)
train_loader_second = DataLoader(train_dataset_second, batch_size=16, shuffle=True)
val_loader_second = DataLoader(val_dataset_second, batch_size=16)

# Обучение модели второго уровня
model_second_level = TextCNN(num_classes=y_second_level.shape[1])
train_model(model_second_level, train_loader_second, val_loader_second, num_epochs=5)

# Предсказания для второго уровня
model_second_level.eval()
predictions_second_level = []
with torch.no_grad():
    for inputs, _ in val_loader_second:
        outputs = model_second_level(inputs)
        predictions_second_level.append((torch.sigmoid(outputs) > 0.4).cpu().numpy())

predictions_second_level = np.vstack(predictions_second_level)

# Фильтрация предсказаний второго уровня на основе первого
def filter_second_level_by_first_level(preds_second_level, preds_first_level, mlb_first_level, mlb_second_level, mapping):
    filtered_second_level_tags = []

    for i in range(len(preds_second_level)):
        predicted_first_tags = mlb_first_level.classes_[preds_first_level[i].astype(bool)]
        allowed_second_level_tags = set()

        for tag in predicted_first_tags:
            if tag in mapping:
                allowed_second_level_tags.update(mapping[tag])

        allowed_indices = mlb_second_level.transform([list(allowed_second_level_tags)])[0].astype(bool)
        filtered_pred = preds_second_level[i] * allowed_indices
        filtered_second_level_tags.append(filtered_pred)

    return np.array(filtered_second_level_tags)

# Фильтрация предсказаний второго уровня
filtered_predictions_second_level = filter_second_level_by_first_level(
    predictions_second_level,
    predictions_first_level,
    mlb_first_level,
    mlb_second_level,
    tag_mapping
)

Epoch 1, Validation Loss: 0.015120897042964185
Epoch 2, Validation Loss: 0.011149261013737746
Epoch 3, Validation Loss: 0.009730520824502622
Epoch 4, Validation Loss: 0.009417096086378609
Epoch 5, Validation Loss: 0.008814775318439518


In [18]:
predicted_first_level_tags = [
    ", ".join(mlb_first_level.classes_[pred.astype(bool)]) for pred in predictions_first_level
]

predicted_second_level_tags = []
for i, pred in enumerate(filtered_predictions_second_level):
    first_level_pred = mlb_first_level.classes_[predictions_first_level[i].astype(bool)]
    second_level_pred = mlb_second_level.classes_[pred.astype(bool)]

    # Создаем строки вида [Первый уровень: Второй уровень]
    combined_tags = []

    for fl_tag in first_level_pred:
        if fl_tag in tag_mapping:  # Если тег первого уровня есть в mapping
            # Получаем соответствующие теги второго уровня
            corresponding_second_level_tags = [f"{fl_tag}: {sl_tag}" for sl_tag in second_level_pred if sl_tag in tag_mapping[fl_tag]]
            combined_tags.extend(corresponding_second_level_tags)

            # Добавляем тег первого уровня, если у него нет соответствующих тегов второго уровня
            if not corresponding_second_level_tags:
                combined_tags.append(fl_tag)

    predicted_second_level_tags.append(combined_tags)

# Создание итогового DataFrame
result_df = pd.DataFrame({
    'video_id': val_video_ids,
    'predicted_tags': predicted_second_level_tags
})

# Устанавливаем video_id как индекс
result_df.set_index('video_id', inplace=True)
result_df


,predicted_tags
video_id,
85c6ac4a4cfc3778cd2e65139d970dba,[Массовая культура]
7eecbd10bdaa596c2921ecb9bf7dbf69,[Массовая культура]
0d7b83b4d141158214acf42ab2ca0519,"[Массовая культура, Религия и духовность]"
cf3ef0b2d6227ad372a9b7dcb6cb2df3,[Массовая культура: Юмор и сатира]
c093daa4f0e96d8f2e4736c9240898f3,"[Семья и отношения, Массовая культура]"
...,...
3a84cf5e54a272f77470dd066f554edb,"[Путешествия, События и достопримечательности]"
484b3600ebf13d5f11b1123b2fe4b142,"[Массовая культура, Спорт]"
d70c45bc09cbbe583c9cede0477040a7,[Массовая культура]


In [19]:
merged_df = data.loc[val_video_ids].merge(result_df, left_index=True, right_index=True, how='inner')
merged_df.reset_index(inplace=True)
merged_df.rename(columns={'index': 'video_id'}, inplace=True)
merged_df

,video_id,title,description,tags,tags_list,first_level_tags,second_level_tags,combined_text,lemmatized_text,combined_vector,predicted_tags
0,85c6ac4a4cfc3778cd2e65139d970dba,"Салют, Начальник | 4 серия",На очередной сельской планёрке обсуждают насущ...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура],[Юмор и сатира],"Салют, Начальник | 4 серия На очередной сельск...",салют начальник 4 серия на очередной сельский ...,"[-0.8496448, -1.2189784, 0.81243056, -0.343441...",[Массовая культура]
1,7eecbd10bdaa596c2921ecb9bf7dbf69,Сколько Стоит Тачка: Галина Ржаксенская - от п...,"В этом выпуске вы узнаете, на чем ездит блогер...",Транспорт: Автомобильная культура,[Транспорт: Автомобильная культура],[Транспорт],[Автомобильная культура],Сколько Стоит Тачка: Галина Ржаксенская - от п...,сколько стоить тачка галина ржаксенский подари...,"[-0.9298673, -1.0066184, 0.47995168, -0.369960...",[Массовая культура]
2,0d7b83b4d141158214acf42ab2ca0519,Какова Меркурия? | Выпуск 12,Женя вместе с астрологом попробуют разобраться...,Религия и духовность: Астрология,[Религия и духовность: Астрология],[Религия и духовность],[Астрология],Какова Меркурия? | Выпуск 12 Женя вместе с аст...,каков меркурий выпуск 12 женя вместе астролог ...,"[-0.39496166, -0.41693857, 0.90392965, -0.3364...","[Массовая культура, Религия и духовность]"
3,cf3ef0b2d6227ad372a9b7dcb6cb2df3,Смехмашина | Выпуск 13,Подвели итоги розыгрыша. Поздравляем счастливо...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура],[Юмор и сатира],Смехмашина | Выпуск 13 Подвели итоги розыгрыша...,смехмашин выпуск 13 подвести итог розыгрыш поз...,"[-1.0671093, -1.4352355, 0.24072486, -0.346048...",[Массовая культура: Юмор и сатира]
4,c093daa4f0e96d8f2e4736c9240898f3,Абьюз-шоу | Выпуск № 9 часть 1 | Ирина Барышева,Героиней сегодняшнего выпуска Абьюз-шоу стала ...,"Семья и отношения: Брак и гражданские союзы, М...","[Семья и отношения: Брак и гражданские союзы, ...","[Семья и отношения, Массовая культура]",[Брак и гражданские союзы],Абьюз-шоу | Выпуск № 9 часть 1 | Ирина Барышев...,абьюз шоу выпуск № 9 часть 1 ирина барышев гер...,"[-0.81049895, -0.672139, 0.46520403, -0.472602...","[Семья и отношения, Массовая культура]"
...,...,...,...,...,...,...,...,...,...,...,...
205,3a84cf5e54a272f77470dd066f554edb,Yunan MotoTour в ГрандТуре «Байкальская миля 2...,Бурятский драматический театр в г. Улан-Удэ.,События и достопримечательности: Театральные м...,[События и достопримечательности: Театральные ...,[События и достопримечательности],[Театральные мероприятия],Yunan MotoTour в ГрандТуре «Байкальская миля 2...,yunan mototour грандтур «байкальский миля 2021...,"[-0.69496167, -0.6221868, 1.1263592, -0.631946...","[Путешествия, События и достопримечательности]"
206,484b3600ebf13d5f11b1123b2fe4b142,Роман Юнусов и популярный озвучер Карен Арутюн...,В новом выпуске шоу «Спортивный Интерес» Рома ...,"Массовая культура, Спорт","[Массовая культура, Спорт]","[Спорт, Массовая культура]",[],Роман Юнусов и популярный озвучер Карен Арутюн...,роман юнус популярный озвучер карен арутюн поп...,"[-0.98206437, -1.0947336, 0.32429215, -0.52538...","[Массовая культура, Спорт]"
207,d70c45bc09cbbe583c9cede0477040a7,PykoJob | Выпуск 4 | Уютная гостиная с небольш...,Ведущая Алиса и дизайнер Наталья показывают ка...,"Хобби и интересы: Мастер-классы, Дом и сад: Ди...","[Хобби и интересы: Мастер-классы, Дом и сад: ...","[Хобби и интересы, Дом и сад]","[Дизайн интерьера, Мастер-классы]",PykoJob | Выпуск 4 | Уютная гостиная с небольш...,pykojob выпуск 4 уютный гостиная небольшой бюд...,"[-0.8206524, -1.85566, 0.5295463, -1.1524012, ...",[Массовая культура]
208,2952b0a7148f5bb09247b22048cd11b9,"Команда ""3/21"" в ГрандТуре-2022: музей В.М. Шу...","""Счастье - оно обыкновенное. Каждый человек ка...","Транспорт, Спорт: Автогонки, События и достопр...","[Транспорт, Спорт: Автогонки, События и дост...","[Спорт, События и достопримечательности, Транс...","[Спортивные собы

In [20]:
merged_df['predicted_tags'] = merged_df['predicted_tags'].astype(str)

In [21]:
merged_df[['video_id', 'predicted_tags']].to_csv('submission.csv')

## Тестирование

In [22]:
import ast

In [23]:
def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    # predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(str)
    predictions_df['predicted_tags'] = predictions_df['predicted_tags'].astype(str)
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(ast.literal_eval)
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        # print(predicted_tags)
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

In [26]:
pred_df = merged_df[['video_id', 'predicted_tags']]
true_df = merged_df[['video_id', 'tags']]

In [27]:
print(find_iou_for_sample_submission(pred_df, true_df))

0.4551247165532879


C:\Users\avo_m\AppData\Local\Temp\ipykernel_1672\255920247.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
C:\Users\avo_m\AppData\Local\Temp\ipykernel_1672\255920247.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))
C:\Users\avo_m\AppData\Local\Temp\ipykernel_1672\255920247.py:31: SettingWithCopyWarning: 
A value is trying to be set o

# Сохранение параметров модели

In [31]:
torch.save(model_first_level.state_dict(), 'model_first_level.pth')
torch.save(model_second_level.state_dict(), 'model_second_level.pth')

In [32]:
import joblib

joblib.dump(mlb_first_level, 'mlb_first_level.pkl')
joblib.dump(mlb_second_level, 'mlb_second_level.pkl')
joblib.dump(tag_mapping, 'tag_mapping.pkl')


['tag_mapping.pkl']